In [1]:
# !jupyter serverextension list
from jupyter_plotly_dash import JupyterDash
###

In [2]:
import pandas as pd
# from tqdm import trange, tqdm
from tqdm import trange, tqdm

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from datetime import datetime, timedelta
from pytz import timezone

import os
import re

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import time
import numpy as np


# Date Functions
def set_date(date_str, old_tz, dt_format = "%d/%m/%Y %H:%M:%S"):
    if date_str == 'NaT':
        return pd.NaT
    else:
        datetime_set_naive = datetime.strptime(date_str, dt_format)
        datetime_set_old = timezone(old_tz).localize(datetime_set_naive)
        datetime_set_utc = datetime_set_old.astimezone(timezone('UTC'))
        return datetime_set_utc

def date_parser(date_, time_, dt_format = "%d/%m/%Y %H:%M:%S"):
    return set_date(date_ + " " + time_, 'UTC', dt_format)


# Experiment start date
date_start = set_date("2019-06-04 13:03:17", "UTC", "%Y-%m-%d %H:%M:%S")
date_now = datetime.now(timezone('UTC'))

# Date Functions continued
def date_range(window=-1, start=date_start, end=date_now):
    if pd.isna(start):
        start = date_start
    if pd.isna(end):
        end = date_now
    if window != -1:
        if not pd.isna(window):
            start = end - timedelta(days=window)
    return start, end

In [3]:
info = pd.read_excel("Info/Info.xlsx", sheet_name=None, index_col=0)
print("info: ", list(info.keys()))

plots = []
for plot, row in info['plots'].iterrows():
    plot_range = date_range(window = row['plot_range_window'], start = set_date(str(row['plot_range_start']), "UTC", "%Y-%m-%d %H:%M:%S"), end = set_date(str(row['plot_range_end']), "UTC", "%Y-%m-%d %H:%M:%S"))
    info['plots'].loc[plot, 'plot_range_start'] = plot_range[0]
    info['plots'].loc[plot, 'plot_range_end'] = plot_range[1]
    info['plots'].loc[plot, 'plot_range_window'] = plot_range[1] - plot_range[0]
    plots.append(plot)

info['plots']

info:  ['setup', 'datasets', 'parameters', 'parameters_ave', 'charts', 'charts_ave', 'plots', 'colours']


,plot_status,plot_res,plot_range_window,plot_range_start,plot_range_end,plot_label
plot,,,,,,
high,ON,0,"7 days, 0:00:00",2020-01-30 11:52:33.674684+00:00,2020-02-06 11:52:33.674684+00:00,High Resolution
med,ON,5,"31 days, 0:00:00",2020-01-06 11:52:33.674684+00:00,2020-02-06 11:52:33.674684+00:00,Medium Resolution
low,ON,120,"246 days, 22:49:16.674684",2019-06-04 13:03:17+00:00,2020-02-06 11:52:33.674684+00:00,Low Resolution
custom1,OFF,0,"1 day, 0:00:00",2019-06-04 13:03:17+00:00,2019-06-05 13:03:17+00:00,Custom Plot


In [4]:
this_dataset_info = info['datasets'].iloc[5]
data_path = info['datasets']['data_folder_path'][5]

info['datasets']

,folder,selection,melt,data_folder_path,supp_data_filepath,file_pat,skiprows,Slim,Del_unit_rows,Add_blank_rows
dataset,,,,,,,,,,
UO,1,UO.wide,UO.wide,C:/Users/Martin/Google Drive/ToOLTuBES/LAB/Dat...,C:/Users/Martin/Google Drive/ToOLTuBES/LAB/Dat...,.csv,0,1.0,NaN,1.0
NWL,1,NWL,NWL,C:/Users/Martin/Google Drive/ToOLTuBES/LAB/Dat...,NaN,.xlsx,2,NaN,1.0,NaN
Skid,1,Skid,Skid.ave,C:/Users/Martin/Google Drive/BEWISE_shared/Ski...,NaN,.CSV,2,NaN,NaN,1.0
Skid,2,Skid,Skid.ave,C:/Users/Martin/Google Drive/BEWISE_shared/Ski...,NaN,.CSV,3,NaN,NaN,1.0
SampLog,1,SampLog.wide,SampLog.wide.ave,C:/Users/Martin/Google Drive/BEWISE_shared/Sam...,NaN,.csv,0,1.0,NaN,NaN
Sensor,1,SENSOR,SENSOR,C:/Users/Martin/Google Drive/ToOLTuBESv2/LAB/T...,NaN,\d\d_\d\d_\d\d\d\d\.csv,0,NaN,NaN,NaN
Events,1,EV.wide,EV.ave,C:/Users/Martin/Google Drive/ToOLTuBESv2/LAB/T...,C:/Users/Martin/Google Drive/ToOLTuBESv2/LAB/T...,.csv,0,NaN,NaN,NaN


In [5]:
info['plots']

,plot_status,plot_res,plot_range_window,plot_range_start,plot_range_end,plot_label
plot,,,,,,
high,ON,0,"7 days, 0:00:00",2020-01-30 11:52:33.674684+00:00,2020-02-06 11:52:33.674684+00:00,High Resolution
med,ON,5,"31 days, 0:00:00",2020-01-06 11:52:33.674684+00:00,2020-02-06 11:52:33.674684+00:00,Medium Resolution
low,ON,120,"246 days, 22:49:16.674684",2019-06-04 13:03:17+00:00,2020-02-06 11:52:33.674684+00:00,Low Resolution
custom1,OFF,0,"1 day, 0:00:00",2019-06-04 13:03:17+00:00,2019-06-05 13:03:17+00:00,Custom Plot


In [6]:
info['parameters']

,code,selected,parameter,parameter_ave,parameter_lab,chart,colours,ribbons,points,fills,ltypes,binary_bar,legend_shows
dataset,,,,,,,,,,,,,
Sensor,A1 V,1,BES_A1__V,BES_1__V_ave,!!BES 1A!!,BES_VA_indiv,0.0,0.0,NaN,NaN,solid,NaN,True
Sensor,A2 V,1,BES_A2__V,BES_2__V_ave,!!BES 2A!!,BES_VA_indiv,3.0,3.0,NaN,NaN,solid,NaN,True
Sensor,A3 V,1,BES_A3__V,BES_3__V_ave,!!BES 3A!!,BES_VA_indiv,5.0,5.0,NaN,NaN,solid,NaN,True
Sensor,A4 V,1,BES_A4__V,BES_4__V_ave,!!BES 4A!!,BES_VA_indiv,6.0,6.0,NaN,NaN,solid,NaN,True
Sensor,B1 V,1,BES_B1__V,BES_1__V_ave,!!BES 1B!!,BES_VB_indiv,0.0,0.0,NaN,NaN,longdash,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Events,HOSE_B_STATUS,0,HOSE_B_STATUS,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
Events,HOSE_C_STATUS,0,HOSE_C_STATUS,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
Events,TILT_SP,0,TILT_SP,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [7]:
info['charts_ave']

,chart,selected,chart_lab,ylab,ymin,ymax,height
order,,,,,,,
1,BES.INDIV_CA,1,BES,Current Density\n(μA/cm²),0,40,10
2,BES.INDIV_CB,1,BES,Current Density\n(μA/cm²),0,40,10
3,BES.INDIV_CC,1,BES,Current Density\n(μA/cm²),0,40,10
4,REXT.INDIV,1,REXT,"R[textstyle(!!Ext!!)],textstyle(!!(!!*Omega*!!...",0,400,8
5,OCP.INDIV,1,OCP,textstyle(!!OCP!!),0,1,6
6,BOD,1,BOD,"atop(textstyle(!!BOD!![5]),textstyle(!!(mg/l O...",0,300,10
7,COD,1,COD,"atop(textstyle(!!COD!!),textstyle(!!(mg/l O!![...",0,600,10
8,TEMP.INDIV,1,Temperature,"atop(textstyle(!!Temperature!!),textstyle(!!(°...",0,60,10
9,TEMP_PROCESS.INDIV,1,TEMP_PROCESS,"atop(textstyle(!!Control!!),textstyle(!!Temp (...",0,150,8


In [8]:
selected_pars = ['A1 C', 'A2 C', 'A3 C', 'A4 C',
                 'B1 C', 'B2 C', 'B3 C', 'B4 C',
                 'C1 C', 'C2 C', 'C3 C', 'C4 C',
                 'TEMP SETPOINT', 'A TEMP', 'B TEMP', 'C TEMP',
                 'BOX TEMP', 'WATER TEMP', 'AMBIENT TEMP',
                 'PUMP FR',
                 'HOSE A TEMP', 'HOSE B TEMP', 'HOSE C TEMP',
                 'MAT 1 TEMP', 'MAT 2 TEMP', 'MAT 3 TEMP',
                 'TILT']

all_days = []

for filename in tqdm(os.listdir(data_path)):
    if re.search(this_dataset_info['file_pat'], filename) and not filename.startswith('.'):
        df = pd.read_csv("".join([data_path, filename]), parse_dates=[['Date', 'Time']], dayfirst=True,
                         date_parser=date_parser)
        df_selected = df[['Date_Time'] + selected_pars]
        all_days.append(df_selected)

all_data = pd.concat(all_days, axis=0, ignore_index=True)
all_data.sort_values(by=['Date_Time'], inplace=True)
# all_data = all_data.set_index('Date_Time')
all_data = all_data.reset_index(drop=True)

100%|██████████| 315/315 [00:31<00:00, 10.02it/s]


In [9]:
plot_dfs = []

for plot in tqdm(plots):
    if info['plots'].loc[plot, 'plot_status'] == 'ON':
        mask = (all_data['Date_Time'] >= info['plots'].loc[plot, 'plot_range_start']) & (all_data['Date_Time'] <= info['plots'].loc[plot, 'plot_range_end'])
        df = all_data.loc[mask]
        if info['plots'].loc[plot, 'plot_res'] != 0:
            df = df.resample("".join([str(info['plots'].loc[plot, 'plot_res']), 'T']), on='Date_Time').mean()
            df = df.reset_index()
        else:
            df = df.reset_index(drop=True)
        plot_dfs.append(df)
    else:
        plot_dfs.append("")

plot_dfs_mlt = []

a = 0
for plot in tqdm(plots):
    if info['plots'].loc[plot, 'plot_status'] == 'ON':
        df = plot_dfs[a].melt(id_vars=['Date_Time'], var_name='Parameter', value_name='Value')
        plot_dfs_mlt.append(df)
    else:
        plot_dfs_mlt.append("")
    a = a + 1

# Assign chart etc
colors = ["red", "orange", "green", "blue",
          "red", "orange", "green", "blue",
          "red", "orange", "green", "blue",
          "black", "red", "green", "blue",
          "cadetblue", "chartreuse", "darkviolet",
          "black",
          "red", "green", "blue",
          "orange", "cyan", "darkviolet",
         "black"]
line_sizes = [1, 1, 1, 1,
              1, 1, 1, 1,
              1, 1, 1, 1,
              1, 1, 1, 1,
              1, 1, 1,
              1,
              1, 1, 1,
              1, 1, 1,
             1]
rows = [1, 1, 1, 1,
        2, 2, 2, 2,
        3, 3, 3, 3,
        4, 4, 4, 4,
        4, 4, 4,
        5,
        6, 6, 6,
        6, 6, 6,
       7]
labels = ["1", "2", "3", "4",
          "1", "2", "3", "4",
          "1", "2", "3", "4",
          "SP", "A", "B", "C",
          "BOX", "WATER", "AMBIENT", 
          "PUMP",
          "HA", "HB", "HC",
          "M1", "M2", "M3",
         "TILT"]
legend_shows = [True, True, True, True,
                False, False, False, False,
                False, False, False, False,
                True, True, True, True,
                True, True, True,
                True,
                True, True, True,
                True, True, True,
               True]

if not os.path.exists("Images"):
    os.mkdir("Images")

figs = []
a = 0

for plot in tqdm(plots):
    if info['plots'].loc[plot, 'plot_status'] == 'ON':
        fig = make_subplots(
            rows=max(rows[0:len(plot_dfs_mlt[a].Parameter.unique())]), cols=1, shared_xaxes=True, vertical_spacing=0.05
        )

        for i in range(0, len(plot_dfs_mlt[a].Parameter.unique())):
            x_data = plot_dfs_mlt[a][plot_dfs_mlt[a].Parameter == plot_dfs_mlt[a].Parameter.unique()[i]].Date_Time
            y_data = plot_dfs_mlt[a][plot_dfs_mlt[a].Parameter == plot_dfs_mlt[a].Parameter.unique()[i]].Value
            name = labels[i]

            fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines',
                                     name=name, line=dict(color=colors[i], width=line_sizes[i]),
                                     connectgaps=True, legendgroup=labels[i], showlegend=legend_shows[i]
                                     ), row=rows[i], col=1)

        # Update yaxis properties
        fig.update_yaxes(title_text="BES A (uA)", range=[0, max(plot_dfs_mlt[a]['Value'])], row=1, col=1)
        fig.update_yaxes(title_text="BES B (uA)", range=[0, max(plot_dfs_mlt[a]['Value'])], row=2, col=1)
        fig.update_yaxes(title_text="BES C (uA)", range=[0, max(plot_dfs_mlt[a]['Value'])], row=3, col=1)
        fig.update_yaxes(title_text="Temp (°C)",  range=[0, 38], row=4, col=1)
        fig.update_yaxes(title_text="Pump FR (ml/min)", range=[0, 33], row=5, col=1)
        fig.update_yaxes(title_text="Heater Temp (°C)", row=6, col=1)
        fig.update_yaxes(title_text="Tilt (°)", row=7, col=1)

        fig.update_layout(
            paper_bgcolor='rgb(255,255,255,0)',
            legend_bgcolor='rgb(255,255,255,0)',
            font=dict(
                family="Arial", size=11, color="#000000"))
        figs.append(fig)
    else:
        figs.append("")
    a = a + 1

a = 0
for plot in tqdm(plots):
    if info['plots'].loc[plot, 'plot_status'] == 'ON':
        figs[a].write_image("Images/fig_" + plot + ".pdf", width=774, height=1052.4, scale=10 / 3)
    a = a + 1

a = 0
for plot in tqdm(plots):
    if info['plots'].loc[plot, 'plot_status'] == 'ON':
        figs[a].write_image("Images/fig_" + plot + ".png", width=800, height=450, scale=5)
    a = a + 1

a = 0
for plot in tqdm(plots):
    if info['plots'].loc[plot, 'plot_status'] == 'ON':
        figs[a].update_layout(
            # height=600,
            # width=900,
            **{"".join(
                ["xaxis", str(max(rows[0:len(plot_dfs_mlt[a].Parameter.unique())])), "_rangeslider_visible"]): True})

        # figs[a].show()
    a = a + 1

# Dash app

dfs = [pd.DataFrame({"xaxis": ["thing", "otherthing", "anotherthing"],
                     "yaxis": [64, 14, 62]}),
       pd.DataFrame({"xaxis": ["newthing", "newotherthing", "newanotherthing"],
                     "yaxis": [344, 554, 112]})]

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

100%|██████████| 4/4 [00:00<00:00, 49.96it/s]


In [10]:
i = 0
output = []
# here you can define your logic on how many times you want to loop
a = 0
for plot in tqdm(plots):
    if info['plots'].loc[plot, 'plot_status'] == 'ON':
        fig = make_subplots(
            rows=max(rows[0:len(plot_dfs_mlt[a].Parameter.unique())]), cols=1, shared_xaxes=True, vertical_spacing=0.05
        )

        for i in range(0, len(plot_dfs_mlt[a].Parameter.unique())):
            x_data = plot_dfs_mlt[a][plot_dfs_mlt[a].Parameter == plot_dfs_mlt[a].Parameter.unique()[i]].Date_Time
            y_data = plot_dfs_mlt[a][plot_dfs_mlt[a].Parameter == plot_dfs_mlt[a].Parameter.unique()[i]].Value
            name = labels[i]

            fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines',
                                     name=name, line=dict(color=colors[i], width=line_sizes[i]),
                                     connectgaps=True, legendgroup=labels[i], showlegend=legend_shows[i]
                                     ), row=rows[i], col=1)

        # Update yaxis properties
        fig.update_yaxes(title_text="BES A (uA)", range=[0, max(plot_dfs_mlt[a]['Value'])], row=1, col=1)
        fig.update_yaxes(title_text="BES B (uA)", range=[0, max(plot_dfs_mlt[a]['Value'])], row=2, col=1)
        fig.update_yaxes(title_text="BES C (uA)", range=[0, max(plot_dfs_mlt[a]['Value'])], row=3, col=1)
        fig.update_yaxes(title_text="Temp (°C)",  range=[0, 38], row=4, col=1)
        fig.update_yaxes(title_text="Pump FR (ml/min)", range=[0, 33], row=5, col=1)
        fig.update_yaxes(title_text="Heater Temp (°C)", row=6, col=1)
        fig.update_yaxes(title_text="Tilt (°)", row=7, col=1)
        
        fig.update_xaxes(range=[min(plot_dfs_mlt[a]['Date_Time']), max(plot_dfs_mlt[a]['Date_Time'])])

        fig.update_layout(
            paper_bgcolor='rgb(255,255,255,0)',
            legend_bgcolor='rgb(255,255,255,0)',
            font=dict(
                family="Arial", size=11, color="#000000"),
            yaxis=dict(
                scaleanchor="x",
                scaleratio=0.2)
            #**{"".join(
            #    ["xaxis", str(max(rows[0:len(plot_dfs_mlt[a].Parameter.unique())])), "_rangeslider_visible"]): True}
            )

        output.append(
            dcc.Graph(id='graph' + str(a),
                      figure=fig,
                      style={'width': '98vw', 'height': '196vh'})
        )
    else:
        output.append("Plot switched off in Info")
    a = a + 1


100%|██████████| 4/4 [00:28<00:00,  7.18s/it]


In [11]:
#app = JupyterDash('__name__')
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

card_style = {
    "box-shadow": "0 4px 5px 0 rgba(0,0,0,0.14), 0 1px 10px 0 rgba(0,0,0,0.12), 0 2px 4px -1px rgba(0,0,0,0.3)"
}

#app.scripts.config.serve_locally = True

# app.layout = dcc.Loading(
#     children=[html.Div(

tabs_init = []
p = 0
for plot in plots:
    if info['plots'].loc[plot, 'plot_status'] == 'ON':
        tabs_init.append(dcc.Tab(label=info['plots']['plot_label'][plot], value="".join(["tab-", str(p)]),
                                            style={'backgroundColor': '#f5f5f5'}))
    p = p+1

app.layout = html.Div(className="sans-serif",
                      children=[
                    html.Div(
                        className="w-60 center pt4",
                        children=[
                            dcc.Tabs(
                                id="tabs",
                                value="tab-0",
                                children=tabs_init,
                                colors={
                                    "primary": "white",
                                    "background": "white",
                                    "border": "#d2d2d2",
                                },
                                parent_style=card_style,
                            ),
                            html.Div(
                                children=[
                                    dcc.Loading(id='tabs-content',
                                                type='graph', className='pv6')
                                ],
                                className='pa4'
                            ),
                        ],
                        style={},
                    ),
                ],
            )# ], type='default', fullscreen=True)

@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    time.sleep(2)
    if tab == 'tab-0':
        return html.Div(children=[
            # html.Label('From 1994 to 2018', id='time-range-label'),
            html.Div(id='loading-0', children=output[0])])
    elif tab == 'tab-1':
        return html.Div(id='loading-1', children=output[1])
    elif tab == 'tab-2':
        return html.Div(id='loading-2', children=output[2])
    elif tab == 'tab-3':
        return html.Div(id='loading-3', children=output[3])

print("App ready: " + str(date_now))

App ready: 2020-02-06 11:52:33.674684+00:00


In [13]:
if __name__ == '__main__':
    app.run_server()
    #debug=True, dev_tools_hot_reload_interval=5000)
                   #dev_tools_hot_reload_max_retry=30)

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Feb/2020 12:11:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 12:11:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 12:11:56] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 12:11:56] "GET /_favicon.ico?v=1.6.1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 12:12:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 12:14:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 12:44:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 12:44:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 12:44:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 12:44:27] "GET /_favicon.ico?v=1.6.1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 12:44:35] "POST /_dash-update-comp

In [21]:
%tb

SystemExit: 1

In [ ]:
plot_info['plot_label']['high']

In [ ]:
daterange = pd.date_range(start='2019',end='2020',freq='W')

def unixTimeMillis(dt):
    ''' Convert datetime to unix timestamp '''
    return int(time.mktime(dt.timetuple()))

def unixToDatetime(unix):
    ''' Convert unix timestamp to datetime. '''
    return pd.to_datetime(unix,unit='s')

def getMarks(start, end, Nth=100):
    ''' Returns the marks for labeling. 
        Every Nth value will be used.
    '''
    result = {}
    for i, date in enumerate(daterange):
        if(i%Nth == 1):
            # Append value to dict
            result[unixTimeMillis(date)] = str(date.strftime('%Y-%m-%d'))
    return result

def get_marks(f, Nth=10000):
    dates = []
    for z in f:
        dates.append(str(z.month) + "-" + str(z.day))
    result = []
    for i, date in enumerate(dates):
        if(i%Nth == 1):
            # Append value to dict
            result.append(date)
    return result

start = set_date(str(row['plot_range_start']), "UTC")
end = set_date(str(row['plot_range_end']), "UTC")
dates_ = plot_dfs_mlt[0][plot_dfs_mlt[0].Parameter == plot_dfs_mlt[0].Parameter.unique()[1]].Date_Time

app = JupyterDash('__name__')

app.layout = html.Div(children=[

html.Label('', id='time-range-label'),
    
dcc.RangeSlider(
    id='date_slider',
    updatemode='mouseup',
    min=0,
    max=len(dates_) - 1,
    count=1,
    step=1,
    value=[0, len(dates_) - 1],
    marks=get_marks(dates_),
),

html.Div(id='loading-0', children=output[0])

])

@app.callback(
    dash.dependencies.Output('time-range-label', 'children'),
    [dash.dependencies.Input('date_slider', 'value')])
def _update_time_range_label(value):
    return '{} --- {}'.format(dates_.iloc[value[0]].strftime("%d/%m/%Y %H:%M"),
                                  dates_.iloc[value[1]].strftime("%d/%m/%Y %H:%M"))

@app.callback(
    Output('loading-0','children'), [Input('date_slider','value')]
)
def update_graph(value): 
    output[0].figure.update_xaxes(range=[dates_.iloc[value[0]], 
                                         dates_.iloc[value[1]]])
    return output[0]

app

In [ ]:
dates_.iloc[1].strftime("%d/%m/%Y, %H:%M")


In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd

import time

app = JupyterDash('__name__')

daterange = pd.date_range(start='1994',end='2018',freq='W')

def unixTimeMillis(dt):
    ''' Convert datetime to unix timestamp '''
    return int(time.mktime(dt.timetuple()))

def unixToDatetime(unix):
    ''' Convert unix timestamp to datetime. '''
    return pd.to_datetime(unix,unit='s')

def getMarks(start, end, Nth=100):
    ''' Returns the marks for labeling. 
        Every Nth value will be used.
    '''

    result = {}
    for i, date in enumerate(daterange):
        if(i%Nth == 1):
            # Append value to dict
            result[unixTimeMillis(date)] = str(date.strftime('%Y-%m-%d'))

    return result

app.layout = html.Div(children=[
    html.H1('Range Slider Testing'),
    html.Div(
        [
            html.Label('From 1994 to 2018', id='time-range-label'),
            dcc.RangeSlider(
                id='year_slider',
                min = unixTimeMillis(daterange.min()),
                max = unixTimeMillis(daterange.max()),
                value = [unixTimeMillis(daterange.min()),
                         unixTimeMillis(daterange.max())],
                marks=getMarks(daterange.min(),
                            daterange.max()),
            ),
        ],
        style={'margin-top': '20'}
    ),
    html.Hr(),

    dcc.Graph(id='my-graph')
])

@app.callback(
    dash.dependencies.Output('time-range-label', 'children'),
    [dash.dependencies.Input('year_slider', 'value')])
def _update_time_range_label(year_range):
    return 'From {} to {}'.format(unixToDatetime(year_range[0]),
                                  unixToDatetime(year_range[1]))

@app.callback(
    dash.dependencies.Output('my-graph', 'figure'),
    [dash.dependencies.Input('year_slider', 'value')])
def _update_graph(year_range):
    date_start = '{}'.format(year_range[0])
    date_end = '{}'.format(year_range[1])

app

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import quandl
import pandas as pd
import datetime
quandl.ApiConfig.api_key = ""

app=JupyterDash('__name__')
#app.css.append_css({'external_url':'https://codepen.io/chriddyp/pen/bWLwgP.css 8'})

df = quandl.get("FRED/GDP")
df= df.reset_index()
df['Date']=pd.to_datetime(df.Date.values)
df['Date']=df["Date"].apply(lambda x: x.year)

unique_years = df.Date.unique()

app.layout = html.Div([
dcc.Graph(id='figure'),

html.Div(
    dcc.RangeSlider(
        id='year-slider',
        min=unique_years[0],
        max=unique_years[-1],
        value=[unique_years[0], unique_years[-1]]
    ),
    style={'padding': '0px 60px'}
)
])

@app.callback(
Output('figure', 'figure'),
[Input('year-slider', 'value')])
def update_figure(selected_years):
    filtered_df = df[
    (df.Date >= selected_years[0]) &
    (df.Date <= selected_years[1])
    ]

    trace = [go.Scatter(
        x=filtered_df.Date,
        y=filtered_df['Value'],
        mode='lines',
        fill='tonexty'
    )]

    return {
        'data': trace,
        'layout': go.Layout(
            title='<b>US GDP over time</b>',
            xaxis={'title': 'Year'},
            yaxis={'title': 'GDP Value'}
        )
    }
app

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots
from plotly import graph_objs as go

fig = make_subplots(rows=2, shared_xaxes=True)
fig.add_scatter(x=[1,2,3], y=[2,1,2])
fig.add_scatter(x=[1,2,3], y=[5,3,3], yaxis='y2')
fig.layout.xaxis.dtick=0.5

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash('__name__')

app.layout = html.Div(children=[

dcc.Graph(
    id='graph1',
    figure = fig
),

dcc.Slider(
    id='temp-slider',
    min=0,
    max=10,
    step=0.2,
    value=0
)

])

@app.callback(
    Output('graph1','figure'), [Input('temp-slider','value')]
)

def update_graph(value): 
    fig.layout.xaxis.range = [value,value+1]
    return fig


app